In [ ]:
pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552450 sha256=a07bdf81604bf333724fe7725409ed03d38b6d1004df48f095ef82ba2a173e8d
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
# Get dataset
# ! pip install datasets
! pip install pandas
import pandas as pd
df = pd.read_csv('jap_data.csv')

In [ ]:
df.head(5)

,Japanese,English
0,生成AIやその他の基盤モデルは、AIの競争原理の変化、 支援技術の新たなレベルへの引き上げ、...,Generative AI and other foundational models ar...
1,本稿に関して、この記事のテーマである生成AI言語モデ ル「ChatGPT」( ※1) に、冒...,"Regarding this paper, I would like to discuss ..."
2,ChatGTP への指示文(プロンプト): 「マッキンゼー・アンド・カンパニーのスタイルで、...,Instructions (prompts) to ChatGTP: “In the sty...
3,ChatGPT の回答: 「生成 AI の台頭は、事業にとって大きなゲームチェンジ ャーとな...,ChatGPT's answer: “The rise of generative AI i...
4,OpenAIが開発したChatGPTに1つのプロンプトを入力す ることで得られる体験は、私た...,Enter a single prompt into ChatGPT developed b...


In [ ]:
# We'll set up the name of the model we want to use, the API key, and the input type.
# Create and retrieve a Cohere API key from dashboard.cohere.ai/welcome/register
# Paste your API key here. Remember to not share publicly
import cohere
model_name = "multilingual-22-12"
input_type_embed = "search_query"

# Now we'll set up the cohere client.
co = cohere.Client('h7BI3irdxYktGqLEpKjfUwOFerutik4FLfXuooTI')

# Get the embeddings
embeds = co.embed(texts=list(df['Japanese']),
                  model=model_name,
                  input_type=input_type_embed).embeddings

In [ ]:
from annoy import AnnoyIndex
import numpy as np
search_index = AnnoyIndex(np.array(embeds).shape[1],'angular')
for i in range(len(embeds)):
    search_index.add_item(i,embeds[i])
search_index.build(10) # 10 trees
search_index.save('test.ann')


True

In [ ]:
query_embed = co.embed(texts=['What is generative AI'],
                  model=model_name,
                  input_type=input_type_embed).embeddings
similar_item_ids = search_index.get_nns_by_vector(query_embed[0], 3, include_distances = True)
results = pd.DataFrame (data={'Texts':df.iloc[similar_item_ids[0]]['Japanese'], 'Distance': similar_item_ids[1],
                              'Eng_Translation':df.iloc[similar_item_ids[0]]['English']})
print(results)

                                               Texts  Distance  \
3  ChatGPT の回答: 「生成 AI の台頭は、事業にとって大きなゲームチェンジ ャーとな...  0.479968   
0  生成AIやその他の基盤モデルは、AIの競争原理の変化、 支援技術の新たなレベルへの引き上げ、...  0.504072   
5  この最新型の生成AIシステムは、多くのトピックを網羅す る、膨大で広範な非構造化データセット...  0.509127   

                                     Eng_Translation  
3  ChatGPT's answer: “The rise of generative AI i...  
0  Generative AI and other foundational models ar...  
5  This state-of-the-art generative AI system cov...  
